In [1]:
import MeCab
mecab = MeCab.Tagger(' -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')
mecab.parse("")  # 追加
from gensim import corpora, matutils, models
import scipy.cluster.hierarchy as hcluster
from sklearn.cluster import KMeans
import collections
from collections import defaultdict

# ↓ 単語辞書（text_clustering_dic.txt）を作成する関数 ↓

In [2]:
def tokenize(text):
    node = mecab.parseToNode(text)
    while node:
        str = node.surface.lower()
        if str != '':
            yield str
        node = node.next
        
# 形態素解析
def keitaiso(text):
    node = mecab.parseToNode(text)
    word=[]
    while node:
        # 単語を取得
        word.append(node.surface)
        # 次の単語に進める
        node = node.next
    return word
        
def get_word_main(content):
    '''一つの記事を形態素解析して返す'''
    return [token for token in tokenize(content)]

def get_words(sentence_list):
    '''記事群のdictについて、形態素解析してリストにして返す'''
    ret = []
    for content in sentence_list:
        ret.append(get_word_main(content))
    return ret

# 辞書生成
def make_dictionary(sentence_list, name):
    words = get_words(sentence_list)
    dictionary = corpora.Dictionary(words)
    dictionary.save_as_text(name)

# ↓ Bag-of-Words ベクトル化してLSIで次元圧縮する関数 ↓

In [3]:
# 全単語数の長さの配列で単語のベクトルを表現
def vec2dense(vec, num_terms):
    return list(matutils.corpus2dense([vec], num_terms=num_terms).T[0])

# lsiで次元圧縮
def lsi_model_gen(bow_list, num_terms):
    lsi_list = []
    lsi_model = models.LsiModel(bow_list, num_topics=num_terms)
    for s in bow_list:
        lsi_list.append(lsi_model[s])

    return lsi_list

def get_bow_list(words_of_each_fund, dictionary):
    bow_list_each_fund = []
    for s in words_of_each_fund:
        bow = dictionary.doc2bow(s)
        bow_list_each_fund.append(bow)
    return bow_list_each_fund

# ↓ クラスタリング ↓

In [4]:
def clustering(each_sentence_list, thresh, dictionary):
    # BoW でベクトル化
    words_of_each_fund = get_words(each_sentence_list) #分かち書きされた単語群を取得
    bow_list_each_fund = get_bow_list(words_of_each_fund, dictionary)

    # LSIを使って次元を5に圧縮してる （例） [0, 1, 1, 0, 1]
    dimension = 5
    lsi_list = lsi_model_gen(bow_list_each_fund, dimension)

    # dimensionの次元の配列としてデータを保存。
    data_all = [vec2dense(l, dimension) for l in lsi_list]

    # thresh = 閾値
    clusters = hcluster.fclusterdata(data_all, thresh, criterion="distance")

    each_sentence_clusters_dict = defaultdict(dict)

    for i in range(len(set(clusters))):
        each_sentence_clusters_dict[i] = []

    for i, j in enumerate(clusters):
        print(i, j)
        new = each_sentence_clusters_dict[j-1]
        new.append(each_sentence_list[i])
        each_sentence_clusters_dict[j-1] = new

    return each_sentence_clusters_dict

# メイン関数

In [5]:
text_list = [
    '“垣根”は相手が作っているのではなく、自分が作っている。',
    '「ああ、ここにおれの進むべき道があった！ようやく掘り当てた！」こういう感投詞を心の底から叫び出される時、貴方がたははじめて心を安んずる事ができるのだろう。',
    '３０分ぐらいでは何もできないと考えているより、世の中の一番つまらぬことでもする方がまさっている。',
    'アイデアは、それを一心に求めてさえいれば必ず生まれる。',
    '貴方がたとえ氷のように潔癖で雪のように潔白であろうとも、世の悪口はまぬがれまい。',
    '貴方がたの人間性を心にとどめ、そして他のことを忘れよ。',
    '貴方と世の中との戦いなら、世の中のほうに賭けなさい。',
    '貴方の遊びに言い訳はいらない。言い訳を決して言うな。',
    'いまの世の中、人間が人間を見捨てているのよね。親が子を、子が親を、兄が弟を、友が友を、隣人が隣人を。',
    'おのれの得るところを軽んずるなかれ。',
    'この世は絶え間のないシーソーだ。',
    'すべては、待っている間に頑張った人のもの。',
]

In [6]:
# 単語辞書作成
make_dictionary(text_list, "text_clustering_dic.txt")
dictionary = corpora.Dictionary.load_from_text("text_clustering_dic.txt")

# クラスタリング
each_sentence_clusters_dict = clustering(text_list, 2.0, dictionary) 

0 2
1 5
2 3
3 1
4 4
5 1
6 1
7 1
8 6
9 1
10 1
11 1


In [7]:
each_sentence_clusters_dict

defaultdict(dict,
            {0: ['アイデアは、それを一心に求めてさえいれば必ず生まれる。',
              '貴方がたの人間性を心にとどめ、そして他のことを忘れよ。',
              '貴方と世の中との戦いなら、世の中のほうに賭けなさい。',
              '貴方の遊びに言い訳はいらない。言い訳を決して言うな。',
              'おのれの得るところを軽んずるなかれ。',
              'この世は絶え間のないシーソーだ。',
              'すべては、待っている間に頑張った人のもの。'],
             1: ['“垣根”は相手が作っているのではなく、自分が作っている。'],
             2: ['３０分ぐらいでは何もできないと考えているより、世の中の一番つまらぬことでもする方がまさっている。'],
             3: ['貴方がたとえ氷のように潔癖で雪のように潔白であろうとも、世の悪口はまぬがれまい。'],
             4: ['「ああ、ここにおれの進むべき道があった！ようやく掘り当てた！」こういう感投詞を心の底から叫び出される時、貴方がたははじめて心を安んずる事ができるのだろう。'],
             5: ['いまの世の中、人間が人間を見捨てているのよね。親が子を、子が親を、兄が弟を、友が友を、隣人が隣人を。']})